In [1]:
import os
import time
from slackclient import SlackClient 
from text_preprocessing import *
from keras.models import load_model

Using TensorFlow backend.


In [2]:
max_story_len = 156
max_query_len = 6

In [3]:
# starterbot's ID as an environment variable
BOT_ID = os.environ.get("BOT_ID")

# constants
AT_BOT = "<@" + BOT_ID + ">"
print(AT_BOT)

<@U5WBCMR88>


In [4]:
model = load_model('model.h5')
text = []
question = ''

In [5]:
# instantiate Slack & Twilio clients
slack_client = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))

In [6]:
def handle_command(command, channel):
    print("command: " + command)
    """
        Receives commands directed at the bot and determines if they
        are valid commands. If so, then acts on the commands. If not,
        returns back what it needs for clarification.
    """
    if command == 'start' or command == "new story":
        text.clear()
        response = "Enter story"
        slack_client.api_call("chat.postMessage", channel=channel, text=response, as_user=True)

    #tokenize question and run on model with text
    elif command.endswith('?'):
        question = tokenize(command)

        vocab = set()
        vocab |= set(text + question)
        vocab = sorted(vocab)

        word_idx = dict((c, i + 1) for i, c in enumerate(vocab))

        story_vector, query_vector = vectorize_ques([(text, question)], word_idx, max_story_len, max_query_len)
        prediction = model.predict([story_vector, query_vector], 32)
        print(prediction)
        if prediction < 0.5:
            response = "No"
        else:
            response = "Yes"
        
        slack_client.api_call("chat.postMessage", channel=channel, text=response, as_user=True)

    #tokenize input and add to text
    elif command != '':
        if not command.endswith('.'):
            command += '.'
        text.extend(tokenize(command))

In [7]:
def parse_slack_output(slack_rtm_output):
    """
        The Slack Real Time Messaging API is an events firehose.
        this parsing function returns None unless a message is
        directed at the Bot, based on its ID.
    """
    output_list = slack_rtm_output
    if output_list and len(output_list) > 0:
        for output in output_list:
            if output and 'text' in output and AT_BOT in output['text']:
                # return text after the @ mention, whitespace removed
                return output['text'].split(AT_BOT)[1].strip(), \
                       output['channel']
    return None, None


In [ ]:
if __name__ == "__main__":
    READ_WEBSOCKET_DELAY = 1 # 1 second delay between reading from firehose
    if slack_client.rtm_connect():
        print("bAbIbot connected and running!")
        while True:
            command, channel = parse_slack_output(slack_client.rtm_read())
            if channel:
                handle_command(command, channel)
            time.sleep(READ_WEBSOCKET_DELAY)
    else:
        print("Connection failed. Invalid Slack token or bot ID?")

bAbIbot connected and running!
command: start
command: Mary moved to the bathroom.


/Applications/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


command: Sandra journeyed to the bedroom.
command: Is Sandra in the hallway?
[[ 0.00856489]]
command: Mary went to the bedroom.
command: Daniel went back to the hallway.
command: Is Daniel in the bathroom?
[[ 0.00080103]]
command: Sandra went to the kitchen.
command: Daniel went back to the bathroom.
command: Is Daniel in the office?
[[ 0.80456084]]
command: Daniel picked up the football there.
command: Daniel went to the bedroom.
command: Is Daniel in the bedroom?
[[ 0.10035319]]
command: start
